In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense,Flatten, LSTM, Conv1D,MaxPooling1D,Dropout,Activation,SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.models import load_model
from keras.callbacks import EarlyStopping
from sklearn import metrics
import nltk
import string
import statistics
from nltk.corpus import stopwords
from sklearn.manifold import TSNE

Using TensorFlow backend.


In [3]:
path="/content/drive/My Drive/Toxic Project/train.csv"
train_data=pd.read_csv(path)
train_data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [4]:
train_data.columns.isna()

array([False, False, False, False, False, False, False, False])

In [0]:
backup = pd.HDFStore('backup_toxic_2.h5')

In [6]:
y_train=np.array(train_data.drop(['id','comment_text'],axis=1))
y_train

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [7]:
path_test="/content/drive/My Drive/Toxic Project/test.csv"
test_data=pd.read_csv(path_test)
test_data

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [8]:
test_label_path="/content/drive/My Drive/Toxic Project/test_labels.csv"
test_label=pd.read_csv(test_label_path)
y_test=test_label.iloc[:,1:7]
y_test[y_test==-1]=1
y_test

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1,1,1,1,1,1
1,1,1,1,1,1,1
2,1,1,1,1,1,1
3,1,1,1,1,1,1
4,1,1,1,1,1,1
...,...,...,...,...,...,...
153159,1,1,1,1,1,1
153160,1,1,1,1,1,1
153161,1,1,1,1,1,1
153162,1,1,1,1,1,1


In [0]:
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def clean_text_sequence(x):
    #clean punctuation 
    x=x.translate(string.punctuation)
    
    #convert text to lower case
    x=x.lower()

    #split the words
    x=x.split()
    
    #remove stop words
    remove_stopwords_list=set(stopwords.words("english"))
    
    #select the threshold
    new_word_list=[word for word in x if not word in  remove_stopwords_list and len(word)>3 ]

            
    text=" ".join(new_word_list)
    
    #clean the text
    #extracted by https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r'\s+', ' ', text)
    
    #remove stemming i.e. overfitting>>>>overfit
    
    text_new=text.split()
    stemmer_type=SnowballStemmer('english')
    stemmed_words=[stemmer_type.stem(word) for word in text_new ]
    
#     for word in text_new:
#         stemmed_words=[]
#         stemmed_words.append(stemmer_type.stem(word))
        
    text_stemmed=" ".join(stemmed_words)
    
    return text_stemmed
    
            
    


In [10]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [0]:
x_train=train_data['comment_text'].map(lambda x:clean_text_sequence(x) )
x_test=test_data['comment_text'].map(lambda x:clean_text_sequence(x) )

In [0]:
unique_words=5000 #maximum number of words for embedding
tokenizer=Tokenizer(num_words=unique_words)
tokenizer.fit_on_texts(x_train)
tokenizer.fit_on_texts(x_test)

In [0]:
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

In [0]:
word_size = len(tokenizer.word_index) + 1   
max_length = 200 #maxmimum number of words for a sequence

In [0]:
x_train_pad = pad_sequences(x_train_seq, padding='post', maxlen=max_length)
x_test_pad = pad_sequences(x_test_seq, padding='post', maxlen=max_length)

In [0]:
path_glove="/content/drive/My Drive/Toxic Project/glove.6B.300d.txt"
embeddings_dict = {}
with open(path_glove, 'r',encoding="utf8") as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

#match the embedding vector into the relavant word index
embedding_word_vector = np.zeros((word_size, 300))
for word, index in tokenizer.word_index.items():
    embedd_vector = embeddings_dict.get(word)
    if embedd_vector is not None:
        embedding_word_vector [index] = embedd_vector

In [17]:
x_train_pad.shape[1]

200

In [0]:
# Embedd_dim=300
# size_of_filter=[3,4]
# no_filters=100

# input=Input(shape=(x_train_pad.shape[1],))
# embedding_layer=Embedding(vocab_size,Embedd_dim,weights=[embedding_word_vector],trainable=True)(input)
# reshape_embedding=Reshape((x_train_pad.shape[1],Embedd_dim,1))(embedding_layer)
# conv_layer1 = Conv2D(no_filters, (size_of_filter.shape[0], Embedd_dim),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape_embedding)
# conv_layer2 = Conv2D(no_filters, (size_of_filter.shape[1], Embedd_dim),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape_embedding)
# maxpool_0 = MaxPooling2D((x_train_pad.shape[1]- size_of_filter.shape[0] + 1, 1), strides=(1,1))(conv_layer1)
# maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)


In [0]:
from keras.layers import Input
from keras.models import Model
embed_inputs = Input(shape=(max_length,))
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_word_vector], trainable=False)(embed_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(6, activation='sigmoid')(LSTM_Layer_1)
model1 = Model(inputs=embed_inputs, outputs=dense_layer_1)

model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 200, 300)          77519400  
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 774       
Total params: 77,739,822
Trainable params: 220,422
Non-trainable params: 77,519,400
_________________________________________________________________


In [0]:
from sklearn.model_selection import train_test_split
x_train1,x_val,y_train1,y_val=train_test_split(x_train_pad,y_train,random_state=42,test_size=0.33)
callbacks = [EarlyStopping(monitor='val_loss')]

In [0]:
model1.fit(x_train1, y_train1, batch_size=400, epochs=10, verbose=2, validation_data=(x_val, y_val),
         callbacks=callbacks)



Train on 106912 samples, validate on 52659 samples
Epoch 1/10
 - 756s - loss: 0.1638 - acc: 0.9618 - val_loss: 0.1398 - val_acc: 0.9630
Epoch 2/10
 - 745s - loss: 0.1398 - acc: 0.9637 - val_loss: 0.1413 - val_acc: 0.9631


In [0]:
model1.save("model1_toxic_2.h5")

In [0]:
from keras.models import load_model
new_model=load_model("model1_toxic_2.h5")

In [0]:
model1.evaluate(x_val, y_val,batch_size=400, verbose=1)

52659/52659 [==============================] - 115s 2ms/step


[0.14131923881656963, 0.9630768952910013]

In [0]:
predict_y_val=model1.predict(x_val)

In [0]:
predict_y_test=model1.predict(x_test_pad)

In [0]:
col_nammes=['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y_validation_val=[]
y_test_val=[]
for i,col in enumerate(col_nammes):
  roc_value_y_validation=metrics.roc_auc_score(y_val[:,i],predict_y_val[:,i])
  roc_value_y_test=metrics.roc_auc_score(y_test[col],predict_y_test[:,i])
  print(col,"Validation_accuracy:",roc_value_y_validation,"\t Test Accuracy:",roc_value_y_test,"\n")
  y_validation_val.append(roc_value_y_validation)
  y_test_val.append(roc_value_y_test)
Accuracy_y_validation=statistics.mean(y_validation_val)
Accuracy_y_test=statistics.mean(y_test_val)
print("Accuracy_y_validation",Accuracy_y_validation)
print("Accuracy_y_test",Accuracy_y_test)



toxic Validation_accuracy: 0.5511206795511311 	 Test Accuracy: 0.5219676795936213 

severe_toxic Validation_accuracy: 0.5460698503002595 	 Test Accuracy: 0.5065598618301134 

obscene Validation_accuracy: 0.5365650667685747 	 Test Accuracy: 0.5132172545126181 

threat Validation_accuracy: 0.497574070324458 	 Test Accuracy: 0.5071377938234055 

insult Validation_accuracy: 0.4858373924203506 	 Test Accuracy: 0.4920984477233238 

identity_hate Validation_accuracy: 0.5093126280178943 	 Test Accuracy: 0.5013044920669169 

Accuracy_y_validation 0.5210799478971113
Accuracy_y_test 0.5070475882583332


In [0]:
model2=Sequential()
seq_length=x_test_pad.shape[1]
model2.add(Embedding(5000, 300, input_length=seq_length))
model2.add(LSTM(256))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(6, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 300)          1500000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 2,087,206
Trainable params: 2,087,206
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.fit(x_train1, y_train1, batch_size=400, epochs=10, verbose=2, validation_data=(x_val, y_val),
         callbacks=callbacks)




Train on 106912 samples, validate on 52659 samples
Epoch 1/10





 - 104s - loss: 0.1468 - acc: 0.9627 - val_loss: 0.1311 - val_acc: 0.9661
Epoch 2/10
 - 96s - loss: 0.1259 - acc: 0.9679 - val_loss: 0.1419 - val_acc: 0.9631


In [0]:
model2.save("model2_toxic_2.h5")

In [0]:
from keras.models import load_model
new_model2=load_model("model2_toxic_2.h5")

In [0]:
new_model2.evaluate(x_val, y_val,batch_size=400, verbose=1)

52659/52659 [==============================] - 15s 281us/step


[0.14190997605888925, 0.9631401992065577]

In [0]:
predict_y_val2=new_model2.predict(x_val)

In [0]:
predict_y_test2=new_model2.predict(x_test_pad)

In [0]:
col_nammes=['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y_validation_val=[]
y_test_val=[]
for i,col in enumerate(col_nammes):
  roc_value_y_validation=metrics.roc_auc_score(y_val[:,i],predict_y_val2[:,i])
  roc_value_y_test=metrics.roc_auc_score(y_test[col],predict_y_test2[:,i])

  print(col,"Validation_accuracy:",roc_value_y_validation,"\t Test Accuracy:",roc_value_y_test,"\n")

  y_validation_val.append(roc_value_y_validation)
  y_test_val.append(roc_value_y_test)
  
Accuracy_y_validation=statistics.mean(y_validation_val)
Accuracy_y_test=statistics.mean(y_test_val)
print("Accuracy_y_validation",Accuracy_y_validation)
print("Accuracy_y_test",Accuracy_y_test)

toxic Validation_accuracy: 0.5192036839548082 	 Test Accuracy: 0.5063329637776801 

severe_toxic Validation_accuracy: 0.5249563506734273 	 Test Accuracy: 0.49939788987057326 

obscene Validation_accuracy: 0.5016465826476991 	 Test Accuracy: 0.49927641810873447 

threat Validation_accuracy: 0.548220826195605 	 Test Accuracy: 0.50621943851948 

insult Validation_accuracy: 0.5023140381683991 	 Test Accuracy: 0.4999507444061633 

identity_hate Validation_accuracy: 0.507104723621296 	 Test Accuracy: 0.499331793973647 

Accuracy_y_validation 0.5172410342102058
Accuracy_y_test 0.5017515414427131


In [0]:
unpolite_word_path="/content/drive/My Drive/Toxic Project/unpolite_word_list.txt"

In [28]:
unpolite_words=pd.read_csv(unpolite_word_path,header=None)
print(unpolite_words)
unpolite_words_list=unpolite_words.iloc[:,0].tolist()
type(unpolite_words_list)

           0
0       4r5e
1       5h1t
2       5hit
3        a55
4       anal
..       ...
446    whore
447  willies
448    willy
449   xrated
450      xxx

[451 rows x 1 columns]


list

In [0]:
word_index = tokenizer.word_index

In [0]:
size=len(unpolite_words_list)
count=0
word_list_temp=unpolite_words_list
for w, index in word_index.items():
    if w in unpolite_words_list:
        word_list_temp.remove(w)
        count+=1 
    if index > (unique_words-size+count):
        for u_w in word_list_temp:
            tokenizer.word_index[u_w] = index
            index+=1
        break

In [0]:
model3=Sequential()
seq_length=x_test_pad.shape[1]
model3.add(Embedding(5000, 300, input_length=seq_length))
model3.add(LSTM(256))
model3.add(Dense(64, activation='relu'))
model3.add(Dense(6, activation='sigmoid'))

model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 300)          1500000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 390       
Total params: 2,087,206
Trainable params: 2,087,206
Non-trainable params: 0
_________________________________________________________________


In [0]:
model3.fit(x_train1, y_train1, batch_size=400, epochs=10, verbose=2, validation_data=(x_val, y_val),
         callbacks=callbacks)

Train on 106912 samples, validate on 52659 samples
Epoch 1/10
 - 99s - loss: 0.1499 - acc: 0.9617 - val_loss: 0.1320 - val_acc: 0.9661
Epoch 2/10
 - 98s - loss: 0.1254 - acc: 0.9679 - val_loss: 0.1240 - val_acc: 0.9680
Epoch 3/10
 - 97s - loss: 0.1193 - acc: 0.9691 - val_loss: 0.1217 - val_acc: 0.9678
Epoch 4/10
 - 97s - loss: 0.1178 - acc: 0.9677 - val_loss: 0.1193 - val_acc: 0.9677
Epoch 5/10
 - 97s - loss: 0.1185 - acc: 0.9676 - val_loss: 0.1414 - val_acc: 0.9632


In [0]:
model3.save("model3_toxic_2.h5")

In [0]:
from keras.models import load_model
new_model3=load_model("model3_toxic_2.h5")

In [0]:
predict_y_val3=new_model3.predict(x_val)

In [0]:
predict_y_test3=new_model3.predict(x_test_pad)

In [0]:
col_nammes=['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y_validation_val=[]
y_test_val=[]
for i,col in enumerate(col_nammes):
  roc_value_y_validation=metrics.roc_auc_score(y_val[:,i],predict_y_val3[:,i])
  roc_value_y_test=metrics.roc_auc_score(y_test[col],predict_y_test3[:,i])

  print(col,"Validation_accuracy:",roc_value_y_validation,"\t Test Accuracy:",roc_value_y_test,"\n")

  y_validation_val.append(roc_value_y_validation)
  y_test_val.append(roc_value_y_test)
  
Accuracy_y_validation=statistics.mean(y_validation_val)
Accuracy_y_test=statistics.mean(y_test_val)
print("Accuracy_y_validation",Accuracy_y_validation)
print("Accuracy_y_test",Accuracy_y_test)

toxic Validation_accuracy: 0.5051185630456401 	 Test Accuracy: 0.5019792036870694 

severe_toxic Validation_accuracy: 0.5193526115093604 	 Test Accuracy: 0.5018964895143468 

obscene Validation_accuracy: 0.5091703627203117 	 Test Accuracy: 0.5026534508744861 

threat Validation_accuracy: 0.5065386018706277 	 Test Accuracy: 0.5006627658053762 

insult Validation_accuracy: 0.5080016770721149 	 Test Accuracy: 0.5025903071990497 

identity_hate Validation_accuracy: 0.5117132661159547 	 Test Accuracy: 0.5015848283625435 

Accuracy_y_validation 0.5099825137223349
Accuracy_y_test 0.5018945075738119


In [34]:
from keras.layers import Input
from keras.models import Model
embed_inputs = Input(shape=(max_length,))
embedding_layer = Embedding(word_size, 300, weights=[embedding_word_vector], trainable=False)(embed_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(6, activation='sigmoid')(LSTM_Layer_1)
model4 = Model(inputs=embed_inputs, outputs=dense_layer_1)

model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 200, 300)          77519400  
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 774       
Total params: 77,739,822
Trainable params: 220,422
Non-trainable params: 77,519,400
_________________________________________________________________


In [35]:
model4.fit(x_train1, y_train1, batch_size=400, epochs=10, verbose=2, validation_data=(x_val, y_val),
         callbacks=callbacks)

Train on 106912 samples, validate on 52659 samples
Epoch 1/10
 - 82s - loss: 0.1662 - acc: 0.9628 - val_loss: 0.1422 - val_acc: 0.9625
Epoch 2/10
 - 81s - loss: 0.1398 - acc: 0.9636 - val_loss: 0.1420 - val_acc: 0.9632
Epoch 3/10
 - 81s - loss: 0.1388 - acc: 0.9639 - val_loss: 0.1268 - val_acc: 0.9632
Epoch 4/10
 - 80s - loss: 0.0846 - acc: 0.9730 - val_loss: 0.0724 - val_acc: 0.9761
Epoch 5/10
 - 81s - loss: 0.0652 - acc: 0.9785 - val_loss: 0.0638 - val_acc: 0.9789
Epoch 6/10
 - 80s - loss: 0.0610 - acc: 0.9795 - val_loss: 0.0641 - val_acc: 0.9790


In [0]:
model4.save("model4_toxic_2.h5")

In [0]:
from keras.models import load_model
new_model4=load_model("model4_toxic_2.h5")

In [39]:
new_model4.evaluate(x_val, y_val,batch_size=400, verbose=1)

52659/52659 [==============================] - 15s 280us/step


[0.06408151398004533, 0.9789652835494461]

In [0]:
predict_y_val4=new_model4.predict(x_val)

In [0]:
predict_y_test4=new_model4.predict(x_test_pad)

In [42]:
col_nammes=['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y_validation_val=[]
y_test_val=[]
for i,col in enumerate(col_nammes):
  roc_value_y_validation=metrics.roc_auc_score(y_val[:,i],predict_y_val4[:,i])
  roc_value_y_test=metrics.roc_auc_score(y_test[col],predict_y_test4[:,i])

  print(col,"Validation_accuracy:",roc_value_y_validation,"\t Test Accuracy:",roc_value_y_test,"\n")

  y_validation_val.append(roc_value_y_validation)
  y_test_val.append(roc_value_y_test)
  
Accuracy_y_validation=statistics.mean(y_validation_val)
Accuracy_y_test=statistics.mean(y_test_val)
print("Accuracy_y_validation",Accuracy_y_validation)
print("Accuracy_y_test",Accuracy_y_test)

toxic Validation_accuracy: 0.9461138434755879 	 Test Accuracy: 0.633812034192025 

severe_toxic Validation_accuracy: 0.9801541888438454 	 Test Accuracy: 0.5755646999416963 

obscene Validation_accuracy: 0.9713371397820441 	 Test Accuracy: 0.6125020380961956 

threat Validation_accuracy: 0.93934048141952 	 Test Accuracy: 0.5732022764133855 

insult Validation_accuracy: 0.9626572731686134 	 Test Accuracy: 0.6084693344116923 

identity_hate Validation_accuracy: 0.9412532015152796 	 Test Accuracy: 0.5785069185851174 

Accuracy_y_validation 0.9568093547008151
Accuracy_y_test 0.597009550273352


In [48]:
from keras.layers import Input
from keras.models import Model
embed_inputs = Input(shape=(max_length,))
embedding_layer = Embedding(word_size, 300, weights=[embedding_word_vector], trainable=False)(embed_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(64, activation='relu')(LSTM_Layer_1)
dense_layer_2 = Dense(6, activation='sigmoid')(dense_layer_1)
model5 = Model(inputs=embed_inputs, outputs=dense_layer_2)

model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 200)               0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 200, 300)          77519400  
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_11 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 390       
Total params: 77,747,694
Trainable params: 228,294
Non-trainable params: 77,519,400
_________________________________________________________________


In [50]:
model5.fit(x_train1, y_train1, batch_size=400, epochs=10, verbose=2, validation_data=(x_val, y_val),
         callbacks=callbacks)

Train on 106912 samples, validate on 52659 samples
Epoch 1/10
 - 82s - loss: 0.1380 - acc: 0.9635 - val_loss: 0.1356 - val_acc: 0.9639
Epoch 2/10
 - 82s - loss: 0.1230 - acc: 0.9658 - val_loss: 0.1141 - val_acc: 0.9640
Epoch 3/10
 - 82s - loss: 0.1090 - acc: 0.9649 - val_loss: 0.1100 - val_acc: 0.9628
Epoch 4/10
 - 82s - loss: 0.1118 - acc: 0.9637 - val_loss: 0.1145 - val_acc: 0.9631


In [0]:
model5.save("model5_toxic_2.h5")

In [0]:
from keras.models import load_model
new_model5=load_model("model5_toxic_2.h5")

In [54]:
new_model5.evaluate(x_val, y_val,batch_size=400, verbose=1)

52659/52659 [==============================] - 15s 277us/step


[0.11448255457241126, 0.9631496944769595]

In [0]:
predict_y_val5=new_model5.predict(x_val)

In [0]:
predict_y_test5=new_model5.predict(x_test_pad)

In [58]:
col_nammes=['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y_validation_val=[]
y_test_val=[]
for i,col in enumerate(col_nammes):
  roc_value_y_validation=metrics.roc_auc_score(y_val[:,i],predict_y_val5[:,i])
  roc_value_y_test=metrics.roc_auc_score(y_test[col],predict_y_test5[:,i])

  print(col,"Validation_accuracy:",roc_value_y_validation,"\t Test Accuracy:",roc_value_y_test,"\n")

  y_validation_val.append(roc_value_y_validation)
  y_test_val.append(roc_value_y_test)
  
Accuracy_y_validation=statistics.mean(y_validation_val)
Accuracy_y_test=statistics.mean(y_test_val)
print("Accuracy_y_validation",Accuracy_y_validation)
print("Accuracy_y_test",Accuracy_y_test)

toxic Validation_accuracy: 0.76818389436878 	 Test Accuracy: 0.5848589417169636 

severe_toxic Validation_accuracy: 0.8688742065756206 	 Test Accuracy: 0.548864111090473 

obscene Validation_accuracy: 0.8226728522592465 	 Test Accuracy: 0.5741719368407189 

threat Validation_accuracy: 0.804092787628316 	 Test Accuracy: 0.5473882939332189 

insult Validation_accuracy: 0.8190379154342197 	 Test Accuracy: 0.5721308221204009 

identity_hate Validation_accuracy: 0.7904996463056887 	 Test Accuracy: 0.5493289285780203 

Accuracy_y_validation 0.8122268837619786
Accuracy_y_test 0.5627905057132992
